In [1]:
import requests
url = "https://herp.careers/careers/jobs"
resp = requests.get(url, timeout=15)
print("Status Code: ", resp.status_code)
print(resp.text[:1000])

Status Code:  200
<!DOCTYPE html><html lang="ja" class="h-full scroll-smooth"><head prefix="og: https://ogp.me/ns#"><meta charSet="utf-8"/><meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1"/><link rel="stylesheet" href="/_next/static/css/d883a9987447c9ae.css" data-precedence="next"/><link rel="preload" as="script" fetchPriority="low" href="/_next/static/chunks/webpack-e8863feb85785555.js"/><script src="/_next/static/chunks/0717c08b-9e47371bac49531b.js" async=""></script><script src="/_next/static/chunks/292-87e4cff6abef8e16.js" async=""></script><script src="/_next/static/chunks/main-app-f9d77102cc7c9099.js" async=""></script><script src="/_next/static/chunks/4675-aecea3eb8c35b13d.js" async=""></script><script src="/_next/static/chunks/5575-99482ebddf14fb37.js" async=""></script><script src="/_next/static/chunks/7798-acce0724fcd725b8.js" async=""></script><script src="/_next/static/chunks/8196-cc43a7421136b77d.js" async=""></script><script src="/_next/

In [ ]:
#Cell2: BeautifulSoupでHTMLをパースし、求人タイトルを抽出
from bs4 import BeautifulSoup
url = "https://herp.careers/careers/jobs"
html = requests.get(url).text
soup = BeautifulSoup(html, "html.parser")

# 求人タイトルを抽出（クラス名の一部で抽出）
job_titles = [
    el.get_text(strip=True)
    for el in soup.find_all("h3")
    if el.get("class") and any("text-[20px]" in c for c in el.get("class"))
]

# 最初の20件を表示
job_titles[:20] 

['株式会社LESS PLUS DESIGN',
 '株式会社ハイウェイ',
 '株式会社Art to Heritage',
 'A1A株式会社',
 'ココザス株式会社',
 '株式会社Conoris Technologies',
 'ROSCA株式会社',
 '株式会社シンカ',
 'REHATCH株式会社',
 '株式会社HACHI',
 'Nature株式会社',
 'エルピクセル株式会社',
 '株式会社GENBA',
 '株式会社Alumnote',
 'NE株式会社',
 '株式会社LabBase',
 'SHE株式会社',
 'クレジットエンジン株式会社',
 'リンクタイズホールディングス株式会社',
 'ちゅらデータ株式会社']

In [3]:
# Cell3: 表示されている1ページ目のみから (company_name, company_url) の組を抽出
import urllib.parse
base_url = "https://herp.careers"
company_pairs = []
seen = set()
# 前のセルで作成した soup (BeautifulSoup オブジェクト) を前提とする
for h3 in soup.find_all('h3'):
    classes = h3.get('class') or []
    if not any('text-[20px]' in c for c in classes):
        continue
    company_name = h3.get_text(strip=True)
    parent_a = h3.find_parent('a', href=True)
    if not parent_a:
        continue
    href = parent_a.get('href')
    if not href:
        continue
    company_url = urllib.parse.urljoin(base_url, href)
    # 企業ページのみを対象にし、重複を除く
    if '/careers/companies/' not in company_url:
        continue
    if company_url in seen:
        continue
    seen.add(company_url)
    company_pairs.append({'company': company_name, 'company_url': company_url})

print(f'Found {len(company_pairs)} unique company pairs on the first page')
company_pairs[:20]
# Check Cell3 output: prints count and sample pairs
try:
    print('Found', len(company_pairs), 'company pairs on the first page')
    company_pairs[:20]
except NameError:
    print('company_pairs not defined yet; run the previous cells to create soup and company_pairs')

print('Company name and URL: ', company_pairs)

Found 20 unique company pairs on the first page
Found 20 company pairs on the first page
Company name and URL:  [{'company': '株式会社LESS PLUS DESIGN', 'company_url': 'https://herp.careers/careers/companies/lpdcoltd?'}, {'company': '株式会社ハイウェイ', 'company_url': 'https://herp.careers/careers/companies/hiway?'}, {'company': '株式会社Art to Heritage', 'company_url': 'https://herp.careers/careers/companies/artheritage?'}, {'company': 'A1A株式会社', 'company_url': 'https://herp.careers/careers/companies/a1aco?'}, {'company': 'ココザス株式会社', 'company_url': 'https://herp.careers/careers/companies/cocozas?'}, {'company': '株式会社Conoris Technologies', 'company_url': 'https://herp.careers/careers/companies/mitsucaru?'}, {'company': 'ROSCA株式会社', 'company_url': 'https://herp.careers/careers/companies/rosca?'}, {'company': '株式会社シンカ', 'company_url': 'https://herp.careers/careers/companies/thinca?'}, {'company': 'REHATCH株式会社', 'company_url': 'https://herp.careers/careers/companies/rehatch?'}, {'company': '株式会社HACHI', '

In [6]:
# Cell4: 2ページ目のみから (company_name, company_url) の組を抽出
import requests
from bs4 import BeautifulSoup
import urllib.parse

url_page2 = "https://herp.careers/careers/jobs?page=2"
resp2 = requests.get(url_page2, timeout=15)
resp2.raise_for_status()
soup2 = BeautifulSoup(resp2.text, 'html.parser')

company_pairs_page2 = []
seen2 = set()
for h3 in soup2.find_all('h3'):
    classes = h3.get('class') or []
    if not any('text-[20px]' in c for c in classes):
        continue
    company_name = h3.get_text(strip=True)
    parent_a = h3.find_parent('a', href=True)
    if not parent_a:
        continue
    href = parent_a.get('href')
    if not href:
        continue
    company_url = urllib.parse.urljoin('https://herp.careers', href)
    if '/careers/companies/' not in company_url:
        continue
    if company_url in seen2:
        continue
    seen2.add(company_url)
    company_pairs_page2.append({'company': company_name, 'company_url': company_url})

print(f'Found {len(company_pairs_page2)} unique company pairs on page 2')
company_pairs_page2[:20]

Found 20 unique company pairs on page 2


[{'company': '株式会社BuySell Technologies',
  'company_url': 'https://herp.careers/careers/companies/buyselltech?'},
 {'company': '株式会社Asobica',
  'company_url': 'https://herp.careers/careers/companies/asobica?'},
 {'company': 'コネクテッドロボティクス株式会社',
  'company_url': 'https://herp.careers/careers/companies/crinc?'},
 {'company': '株式会社KINS',
  'company_url': 'https://herp.careers/careers/companies/yourkins?'},
 {'company': 'イタンジ株式会社',
  'company_url': 'https://herp.careers/careers/companies/itandi?'},
 {'company': '株式会社Magic Shields',
  'company_url': 'https://herp.careers/careers/companies/magicshields?'},
 {'company': '株式会社GenerativeX',
  'company_url': 'https://herp.careers/careers/companies/generativex?'},
 {'company': '株式会社UMITO',
  'company_url': 'https://herp.careers/careers/companies/umito?'},
 {'company': '株式会社HUMAN LIFE',
  'company_url': 'https://herp.careers/careers/companies/humanlife?'},
 {'company': '株式会社ビットエー',
  'company_url': 'https://herp.careers/careers/companies/bita?'},
 

In [16]:
# Cell5: 1ページ目から page=番号 を使って最後のページまで会社名とリンクを抽出
import time
import random
import requests
from bs4 import BeautifulSoup
import urllib.parse
from urllib.parse import urlparse, urlunparse
import re

base = 'https://herp.careers'
all_companies = []
seen = set()
consecutive_no_pairs = 0
# 連続して added==0 の場合に停止するためのカウンタ
consecutive_no_new_added = 0
stop_after_no_new_added = 2
max_pages = 200

def normalize_url(u):
    p = urlparse(u)
    # path を正規化（末尾スラッシュを削除）し、クエリやフラグメントを廃棄する
    path = p.path.rstrip('/')
    return urlunparse((p.scheme, p.netloc, path, '', '', ''))

# まず一度サイトにアクセスして 8〜12秒待つ（サーバー負荷を下げるため）
try:
    r0 = requests.get(f'{base}/careers/jobs', timeout=15)
    r0.raise_for_status()
    print('Initial fetch ok, sleeping 8-12s before scraping')
    time.sleep(random.uniform(8, 12))
except Exception as e:
    print('Initial fetch failed:', e)

# page=N の方式で順に取得する（Cell4 と同様の抽出ロジック）
for page in range(1, max_pages + 1):
    url = f'{base}/careers/jobs?page={page}'
    try:
        r = requests.get(url, timeout=15)
        r.raise_for_status()
    except Exception as e:
        print('Request failed on page', page, ':', e)
        break
    soup = BeautifulSoup(r.text, 'html.parser')
    pairs = []
    for h3 in soup.find_all('h3'):
        classes = h3.get('class') or []
        if not any('text-[20px]' in c for c in classes):
            continue
        name = h3.get_text(strip=True)
        parent_a = h3.find_parent('a', href=True)
        if not parent_a:
            continue
        href = parent_a.get('href')
        if not href:
            continue
        comp_url = urllib.parse.urljoin(base, href)
        if '/careers/companies/' not in comp_url:
            continue
        comp_url = normalize_url(comp_url)
        pairs.append((name, comp_url))
    print(f'page {page}: pairs_found={len(pairs)}')
    if not pairs:
        consecutive_no_pairs += 1
        if consecutive_no_pairs >= 3:
            print('Stopping after', consecutive_no_pairs, 'consecutive empty pages')
            break
        time.sleep(0.2)
        continue
    consecutive_no_pairs = 0
    added = 0
    for name, comp_url in pairs:
        if comp_url not in seen:
            seen.add(comp_url)
            all_companies.append({'company': name, 'company_url': comp_url})
            added += 1
    if added == 0:
        consecutive_no_new_added += 1
        print('All pairs on this page were duplicates (added==0). Sample URLs and whether seen (normalized):')
        sample = [u for _, u in pairs][:10]
        for u in sample:
            print('  ', u, 'in seen=', u in seen)
        if consecutive_no_new_added >= stop_after_no_new_added:
            print('Stopping: no new companies for', consecutive_no_new_added, 'pages')
            break
    else:
        consecutive_no_new_added = 0
    print(f'page {page}: pairs={len(pairs)} added={added} total={len(all_companies)}')
    time.sleep(0.2)

print('Collected', len(all_companies), 'unique companies')
if all_companies:
    print('\nFirst 10:')
    for c in all_companies[:10]:
        print(c)
    print('\nLast 10:')
    for c in all_companies[-10:]:
        print(c)

# 表示確認用に最初/最後を返す
all_companies[:10] + all_companies[-10:] if all_companies else []

Initial fetch ok, sleeping 8-12s before scraping
page 1: pairs_found=20
page 1: pairs=20 added=20 total=20
page 2: pairs_found=20
page 2: pairs=20 added=20 total=40
page 3: pairs_found=20
page 3: pairs=20 added=20 total=60
page 4: pairs_found=20
page 4: pairs=20 added=20 total=80
page 5: pairs_found=20
page 5: pairs=20 added=20 total=100
page 6: pairs_found=20
page 6: pairs=20 added=20 total=120
page 7: pairs_found=20
page 7: pairs=20 added=20 total=140
page 8: pairs_found=20
page 8: pairs=20 added=20 total=160
page 9: pairs_found=20
page 9: pairs=20 added=13 total=173
page 10: pairs_found=20
page 10: pairs=20 added=19 total=192
page 11: pairs_found=20
page 11: pairs=20 added=17 total=209
page 12: pairs_found=20
page 12: pairs=20 added=7 total=216
page 13: pairs_found=20
page 13: pairs=20 added=20 total=236
page 14: pairs_found=20
page 14: pairs=20 added=20 total=256
page 15: pairs_found=20
page 15: pairs=20 added=20 total=276
page 16: pairs_found=20
page 16: pairs=20 added=20 total=29

[{'company': '株式会社LESS PLUS DESIGN',
  'company_url': 'https://herp.careers/careers/companies/lpdcoltd'},
 {'company': '株式会社ハイウェイ',
  'company_url': 'https://herp.careers/careers/companies/hiway'},
 {'company': '株式会社Art to Heritage',
  'company_url': 'https://herp.careers/careers/companies/artheritage'},
 {'company': 'A1A株式会社',
  'company_url': 'https://herp.careers/careers/companies/a1aco'},
 {'company': 'ココザス株式会社',
  'company_url': 'https://herp.careers/careers/companies/cocozas'},
 {'company': '株式会社Conoris Technologies',
  'company_url': 'https://herp.careers/careers/companies/mitsucaru'},
 {'company': 'ROSCA株式会社',
  'company_url': 'https://herp.careers/careers/companies/rosca'},
 {'company': '株式会社シンカ',
  'company_url': 'https://herp.careers/careers/companies/thinca'},
 {'company': 'REHATCH株式会社',
  'company_url': 'https://herp.careers/careers/companies/rehatch'},
 {'company': '株式会社HACHI',
  'company_url': 'https://herp.careers/careers/companies/hachi'},
 {'company': 'MyDearest株式会社',